In [6]:
# source_path = r'C:\Users\ping\Downloads\df_finviz_2025-04-05.parquet'
source_path = r'C:\Users\ping\Downloads\df_OHLCV_2025-04-05.parquet'

In [7]:
import pandas as pd
df = pd.read_parquet(source_path)
df

Open    High     Low   Close  Adj Close   Volume  \
Symbol Date                                                             
DXJ    2025-04-04   97.00   97.68   94.69   96.08      96.08  1730700   
       2025-04-03  103.45  103.64  101.63  102.26     102.26  1232600   
       2025-04-02  107.73  109.70  107.67  109.60     109.60   237600   
       2025-04-01  108.75  109.60  107.81  109.25     109.25   656700   
       2025-03-31  109.39  110.38  108.76  110.18     110.18   401100   
...                   ...     ...     ...     ...        ...      ...   
FV     2024-04-11   56.08   56.47   55.63   56.40      56.27    65100   
       2024-04-10   55.60   56.05   55.51   55.64      55.51    85600   
       2024-04-09   56.73   56.73   56.01   56.70      56.57    79200   
       2024-04-08   56.39   56.56   56.21   56.30      56.17    84300   
       2024-04-05   55.74   56.42   55.70   56.18      56.05    97100   

                     Adj Open    Adj High     Adj Low  
Symbol Date                                            
DXJ    2025-04-04   97.000000   97.680000   94.690000  
       2025-04-03  103.450000  103.640000  101.630000  
       2025-04-02  107.730000  109.700000  107.670000  
       2025-04-01  108.750000  109.600000  107.810000  
       2025-03-31  109.390000  110.380000  108.760000  
...                       ...         ...         ...  
FV     2024-04-11   55.950738   56.339839   55.501775  
       2024-04-10   55.470093   55.919042   55.380304  
       2024-04-09   56.599931   56.599931   55.881582  
       2024-04-08   56.259792   56.429400   56.080208  
       2024-04-05   55.611018   56.289445   55.571111  

[753 rows x 9 columns]

In [ ]:
        
def calculate_weighted_score(df):
    components = {}
    intermediate_values = {}
    raw_values = {}

    # 1. Store raw ATR/Price values
    raw_values['atr_price_raw'] = df['ATR/Price %'] if 'ATR/Price %' in df.columns else pd.Series(0, index=df.index)

    # 2. Risk-adjusted metrics
    for category, cols in [('sharpe', sharpe_cols),
                          ('sortino', sortino_cols),
                          ('omega', omega_cols)]:
        z_scores = df[cols].apply(lambda x: (x - x.mean()) / x.std())
        intermediate_values[f'{category}_zscores'] = z_scores
        raw_values[f'{category}_raw'] = df[cols]
        components[category] = z_scores.mean(axis=1) * feature_weights[category]

    # 3. Enhanced Volatility Calculation
    volatility_zscores = df[volatility_cols].apply(lambda x: (x - x.mean()) / x.std())
    intermediate_values['volatility_zscores'] = volatility_zscores
    raw_values['volatility_raw'] = df[volatility_cols]
    
    # Debug: Individual weighted components
    for col in volatility_cols:
        intermediate_values[f'volatility_{col}_weighted'] = (
            volatility_zscores[col] * feature_weights['volatility'] / len(volatility_cols))
    
    components['volatility'] = volatility_zscores.mean(axis=1) * feature_weights['volatility']

    # 4. Momentum (keep your existing implementation)
    momentum_zscores = df[momentum_cols].apply(lambda x: (x - x.mean()) / x.std())
    intermediate_values['momentum_zscores'] = momentum_zscores
    raw_values['momentum_raw'] = df[momentum_cols]
    components['momentum'] = (momentum_zscores @ momentum_weights) * feature_weights['momentum']

    # 5. RSI (keep your enhanced RSI scoring)
    raw_values['rsi_raw'] = df['RSI']
    if 'RSI' in df.columns:
        rsi_score = pd.Series(
            np.select(
                [df['RSI'] > 70, df['RSI'] < 30, (df['RSI'] >= 30) & (df['RSI'] <= 70)],
                [np.clip((df['RSI'] - 70) * -0.01, -0.15, 0),
                 np.clip((30 - df['RSI']) * 0.005, 0, 0.075),
                 0]),
            index=df.index)
        components['rsi'] = rsi_score
    else:
        components['rsi'] = pd.Series(0, index=df.index)

    # 6. SMA (unchanged)
    sma_zscores = df[sma_cols].apply(lambda x: (x - x.mean()) / x.std())
    intermediate_values['sma_zscores'] = sma_zscores
    raw_values['sma_raw'] = df[sma_cols]
    components['sma'] = sma_zscores.mean(axis=1) * feature_weights['sma']

    # Final composite assembly
    component_breakdown = pd.concat(components, axis=1)
    component_breakdown.columns = [f"weighted_{col}" for col in component_breakdown.columns]
    intermediate_values['component_breakdown'] = component_breakdown
    
    return component_breakdown.sum(axis=1), intermediate_values, raw_values



In [1]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils
from config import date_str, DOWNLOAD_DIR, DEST_DIR

path_data = '..\data\df_finviz_n_ratios_merged.parquet'
path_corr = '..\data\df_corr_emv_matrix.parquet'
path_cov = '..\data\df_cov_emv_matrix.parquet'
path_output = f'..\picks\{date_str}_portf.txt'

print(utils.__file__)  # Should point to your src/utils.py
print(f'path_date: {path_data}')
print(f'path_corr: {path_corr}')
print(f'path_cov: {path_cov}')  
print(f'path_output: {path_output}')
print((f'date_str: {date_str}'))



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']
c:\Users\ping\Files_win10\python\py311\stocks\src\utils.py
path_date: ..\data\df_finviz_n_ratios_merged.parquet
path_corr: ..\data\df_corr_emv_matrix.parquet
path_cov: ..\data\df_cov_emv_matrix.parquet
path_output: ..\picks\2025-04-03_portf.txt
date_str: 2025-04-03


In [2]:
print(list((ROOT_DIR / 'src').glob('*.py')))

[WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/src/utils.py'), WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/src/__init__.py')]


In [3]:
import sys
print(sys.path)  # Check if ROOT_DIR / 'src' is listed

['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\setuptools\\_vendor']


In [4]:
import utils
print(utils.__file__)  # Should point to your src/utils.py

c:\Users\ping\Files_win10\python\py311\stocks\src\utils.py
